---
title: "2023_Kronensicherung_Plesse_001_Versuchsablauf"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Kraefte_Schwingungen

## Baumdaten, Gerädeanordung und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installiere die Pakete linescale3 (LS3) und treeqinetic (PTQ) inklusive kj_core und kj_logger und weiteren requirements.

## Arbeitsumgebung vorbereiten

Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend die zwei extra geschriebenen Pakete LS3 und PTQ. Fehler beim Import dieser zwei Pakete sind ggf. Bugs. Beide Pakete nutzen eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [ ]:
# Struktur
import json
from pathlib import Path

# Datenverarbeitung
import numpy as np
import pandas as pd
from IPython.display import Markdown, display
from pandas.api.types import CategoricalDtype
from slugify import slugify  # Slugify ums strings in standard Formate zu überführen

Lade allgemeine Export-Funktionen, um die Daten als Latex-Tabellen zu exportieren

In [ ]:
from kj_core.utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

### IMPORT: Importieren von eigenen Modulen

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [ ]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    analyse_name,
    data_path,
    working_directory,
    data_export_directory,
    latex_export_directory
)



## Vorbemerkung

Die Daten wurden während des Versuchs als GoogleSheet erfasst und abschließend als CSV-Dateien exportiert, um ein einfaches Einlesen zur ermöglichen.
Nachfolgend werden Datentypen für alle Spalten explizit in den *_data_dict.json angegeben, da es teils durch Python-Pandas zu Fehlerkennungen kommt.
Zusätzlich werden teils Spaltennamen vereinheitlicht oder benannt.

## Daten des Versuchsbaums (tree_df)

### IMPORT: Daten und Datendokumentation

Die `tree.csv` liefert spezifische Informationen über den Versuchsbaum. Da nur ein Versuchsbaum,sind hier Daten und Datendokumentation in einer Datei.

In [ ]:
tree_df = pd.read_csv(
    data_path / 'tree.csv',
    sep=';', decimal=',',
    na_values='NA',
    dtype='string'  # alle Spalten als string
)

### ANALYSE: Explorative Datenanalyse

In [ ]:
#tree_df

In [ ]:
# In Markdown umwandeln und anzeigen
md_text = tree_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv)

In [ ]:
# DataFrame als Feather
tree_df.to_feather(data_export_directory / "tree.feather")
tree_df.to_csv(data_export_directory / "tree.csv", sep=";", index=False, encoding="utf-8")

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [ ]:
select_data_fields = ["Zeichen", "Deutsch", "Wert", "Datentyp", "Einheit"]
tree_df_latex = tree_df[select_data_fields].copy()

latex_string = tree_df_latex.to_latex(index=False, escape=False, column_format="llrrr",
                                      float_format="{:0.2f}".format)
caption = "Plesse - Daten des Versuchsbaums"

save_latex_table(latex_string, caption, latex_export_directory)

## Daten der Geräteanordnung am Baum (sensor_df)

### IMPORT: Daten und Datendokumentation

Die Datendokumentation ergibt sich aus `sensor_position_data_dict.json`.

Die `sensor_position.csv` enthält detaillierte Informationen zur Anordnung der Sensoren an den Bäumen im Rahmen des Experiments. Jede Zeile beschreibt die Platzierung eines Sensors.

In [ ]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path/ "sensor_position_data_dict.json", "r", encoding="utf-8") as f:
    sensor_data_dict = json.load(f)

In [ ]:
# Erzeuge dtype_dict dynamisch aus dem der Daten Dokumentation
dtype_dict = {key: value["Datentyp"] for key, value in sensor_data_dict.items()
    if value["Datentyp"] not in [None, ""]
}

# CSV einlesen mit dynamischen Datentypen
sensor_file = data_path / 'sensor_position.csv'
sensor_df = pd.read_csv(sensor_file, sep=';', decimal=',', na_values='NA',dtype=dtype_dict)

### ANALYSE: Explorative Datenanalyse

In [ ]:
# In DataFrame umwandeln
sensor_data_dict_df = build_data_dict_df(sensor_data_dict)

# In Markdown umwandeln und anzeigen
md_text = sensor_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

In [ ]:
sensor_df["height"] = sensor_df["height"] / 100
sensor_df["circumference"] = sensor_df["circumference"] / 100
sensor_df["diameter"] = sensor_df["circumference"] / np.pi

sensor_df.head()

### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [ ]:
# DataFrame als Feather
sensor_df.to_feather(data_export_directory / "sensor.feather")
sensor_df.to_csv(data_export_directory / "sensor.csv", sep=";", index=False, encoding="utf-8")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "sensor_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(sensor_data_dict, f,  indent=4, ensure_ascii=False)

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [ ]:
sensor_select_variables = ["type", "sensor_id", "location", "direction", "height", "circumference", "diameter"]

In [ ]:
sensor_df_latex = sensor_df.copy()[sensor_select_variables]

# Ersetze Spaltennamen durch Formelzeichen
sensor_df_latex.columns = [sensor_data_dict[col]["Zeichen"] for col in sensor_select_variables]

latex_string = sensor_df_latex.to_latex(index=False, escape=False, column_format="lllrrrr",
    float_format="{:0.2f}".format)

caption = "Plesse - Geräteanordnung"
save_latex_table(latex_string, caption, latex_export_directory)

In [ ]:
sensor_data_dict_df = build_data_dict_df(sensor_data_dict, keys=sensor_select_variables, escape_index=True, select_latex_fields=True)
latex_string = sensor_data_dict_df.to_latex(index=False, escape=False)
caption = "Plesse - Geräteanordnung Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

## Daten des Versuchsablaufs (series_df)

### IMPORT: Daten und Datendokumentation

Die Datendokumentation ergibt sich aus `series_data_dict.json`.

Die `series.csv` enthält das Protokoll des Versuchsablaufes der einzelnen Messungen. Jede Zeile repräsentiert eine einzelne Messung. 

In [ ]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path/ "series_data_dict.json", "r", encoding="utf-8") as f:
    series_data_dict = json.load(f)

In [ ]:
# Erzeuge dtype_dict dynamisch aus dem der Daten Dokumentation
dtype_dict = {key: value["Datentyp"] for key, value in series_data_dict.items()
    if value["Datentyp"] not in [None, ""]
}

# CSV einlesen mit dynamischen Datentypen
series_file = data_path / 'series.csv'
series_df = pd.read_csv(series_file, sep=';', decimal=',', na_values='NA', dtype=dtype_dict)

# Zeitspalte in Uhrzeit umwandeln
series_df["time"] = pd.to_datetime(series_df["time"], format="%H:%M:%S").dt.time

In [ ]:
# Behandlungskategorien sortiert definieren
treatment_order = ['free', 'gefa_dynamic', 'cobra_static', 'cobra_static_slack']
treatment_category = CategoricalDtype(categories=treatment_order, ordered=True)
series_df["treatment"] = series_df["treatment"].astype(treatment_category)

### ANALYSE: Explorative Datenanalyse

Behandlungsvariante/Kronensicherung Kategorien:

- **`free`**: Der Baum konnte frei ohne Kronensicherung nach dem Release ausschwingen.
  
- **`gefa_dynamic`**: In ca. 2/3 der Baumhöhe wurde ein dynamisches Gefa Gurtband 4t dynamisch nach ZTV-Baumpflege mit leichtem Durchhang installiert. Das Ausschwingen wurde durch die KS abgedämpft. Da die Sicherung ohne Vorspannung installiert wurde, zeigen die Plots (/ls3/plots/force_vs_time_1/) von '14:99:1E' sowohl am Anfang als auch am Ende ca. 0 kN Kraft an (LogNr 1 bis 9).

- **`cobra_static`**: In ca. 2/3 der Baumhöhe wurde eine statische Cobra ultrastatic 7t (Dyneema) Sicherung installiert. Die Vorspannung betrug ca. 0,4 kN, wie sich in den Plots (/ls3/plots/force_vs_time_1/) von '14:99:1E' gut erkennen lässt. Durch das Zusammenziehen der Stämmlinge ist die Kronensicherung vor dem Release vollständig lastfrei (0 kN). Nach dem Release pendelt sich die Kraft ca. bei 0,4 kN ein (LogNr 10-18).

- **`cobra_static_slack`**: Ähnlich der `cobra_static`, jedoch wurde die Vorspannung entfernt. Aufgrund von Regen wurde nur eine Messung durchgeführt und die Serie frühzeitig abgebrochen.


In [ ]:
# In DataFrame umwandeln
series_data_dict_df = build_data_dict_df(series_data_dict)

# In Markdown umwandeln und anzeigen
md_text = series_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

In [ ]:
series_df.head(50)

### EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [ ]:
# DataFrame als Feather
series_df.to_feather(data_export_directory / "series.feather")
series_df.to_csv(data_export_directory / "series.csv", sep=";", index=False, encoding="utf-8")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "series_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(series_data_dict, f,  indent=4, ensure_ascii=False)

### LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [ ]:
series_select_variables = ["id", "time", "release_force_target", "treatment", "14:BF:E6", "14:99:1E"]

In [ ]:
series_df_latex = series_df.copy()[series_select_variables]

# Ersetze Spaltennamen durch Formelzeichen
series_df_latex.columns = [series_data_dict[col]["Zeichen"] for col in series_select_variables]

series_df_latex["treatment"] = series_df_latex["treatment"].apply(
    lambda x: "\\texttt{" + str(x).replace("_", "\\_") + "}"
)

latex_string = series_df_latex.to_latex(index=False, escape=False, column_format="llrlrr",
                                        float_format="{:0.2f}".format)

caption = "Plesse - Versuchsablauf"
caption_long = "Plesse - Versuchsablauf"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)

In [ ]:
series_data_dict_df = build_data_dict_df(series_data_dict, keys=series_select_variables, escape_index=True, select_latex_fields=True)

latex_string = series_data_dict_df.to_latex(index=False, escape=False)
caption = "Plesse - Versuchsablauf Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)